In [2]:
import h2o
import pandas as pd
from h2o.automl import H2OAutoML
h2o.init()



Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 mins 53 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.6
H2O_cluster_version_age:,20 days
H2O_cluster_name:,H2O_from_python_mimivelloso_q3qpbm
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.767 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [5]:
train =  pd.read_csv("output/trainclean.csv", index_col=0)
test = pd.read_csv("output/testclean.csv", index_col=0)

In [13]:
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [14]:
x = train.columns
y = "price"
x.remove(y)


In [16]:
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [20]:
b = aml.leaderboard
b.head() 

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200720_195209,293835,542.065,293835,285.794,nan
StackedEnsemble_BestOfFamily_AutoML_20200720_195209,293928,542.152,293928,283.628,nan
GBM_1_AutoML_20200720_195209,306813,553.907,306813,296.816,nan
GBM_2_AutoML_20200720_195209,309007,555.884,309007,302.496,nan
XGBoost_grid__1_AutoML_20200720_195209_model_4,309666,556.476,309666,295.281,nan
GBM_3_AutoML_20200720_195209,309908,556.694,309908,300.643,nan
GBM_4_AutoML_20200720_195209,314833,561.1,314833,310.375,0.143649
GBM_5_AutoML_20200720_195209,324152,569.343,324152,305.248,nan
XGBoost_grid__1_AutoML_20200720_195209_model_3,330434,574.834,330434,312.686,nan
XGBoost_grid__1_AutoML_20200720_195209_model_1,342397,585.147,342397,329.192,nan


In [23]:
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20200720_195209

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 201498.30056755207
RMSE: 448.8856208072966
MAE: 247.82823702312677
RMSLE: 0.10369518445131935
R^2: 0.9873940482948309
Mean Residual Deviance: 201498.30056755207
Null degrees of freedom: 10037
Residual degrees of freedom: 10031
Null deviance: 160451365035.91312
Residual deviance: 2022639941.0970876
AIC: 151102.08663347494

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 293834.8545356923
RMSE: 542.0653600219187
MAE: 285.7938104445249
RMSLE: NaN
R^2: 0.981497006167357
Mean Residual Deviance: 293834.8545356923
Null degrees of freedom: 40454
Residual degrees of freedom: 40448
Null deviance: 642466441762.4946
Residual deviance: 11887089040.241432
AIC: 624182.045282834


In [29]:
preds = aml.leader.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [30]:
preds

predict
15277.4
15042.7
1017.03
437.018
3563.88
1906.5
802.017
5293.14
717.468
4183.26


In [49]:
test = preds.as_data_frame()
test.rename(columns = {"predict":"price"}, inplace=True)
test.index.rename("id", inplace=True)
test

,price
id,
0,15277.365949
1,15042.702480
2,1017.026268
3,437.018208
4,3563.884742
...,...
13480,2583.877296
13481,5084.887434
13482,2043.744905


In [50]:
test.to_csv("output/test6.csv")